# DimUser

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *





In [0]:
import os
import sys
project_path = os.path.join(os.getcwd(),'..','..')
sys.path.append(project_path)
from utils.transform import reusable

### AUTOLOADER


In [0]:
df_user = spark.readStream\
  .format("cloudFiles")\
  .option("cloudFiles.format", "parquet")\
  .option("cloudFiles.schemaLocation", "abfss://silver@storagespotifyproject96.dfs.core.windows.net/DimUser/checkpoint")\
  .load("abfss://bronze@storagespotifyproject96.dfs.core.windows.net/DimUser")

In [0]:
%sql
GRANT READ FILES ON EXTERNAL LOCATION `bronze` TO `harshavardhan96035_gmail.com#ext#@harshavardhan96035gmail.onmicrosoft.com`; GRANT READ FILES ON EXTERNAL LOCATION `silver` TO `harshavardhan96035_gmail.com#ext#@harshavardhan96035gmail.onmicrosoft.com`; 
GRANT WRITE FILES ON EXTERNAL LOCATION `silver` TO `harshavardhan96035_gmail.com#ext#@harshavardhan96035gmail.onmicrosoft.com`;

In [0]:
display(df_user)

In [0]:
df_user = df_user.withColumn("user_name",upper(col("user_name")))



In [0]:
df_user_obj  = reusable()
df_user = df_user_obj.dropColumns(df_user,['_rescued_data'])
df_user = df_user.dropDuplicates(['user_id'])


In [0]:
display(df_user)

In [0]:
df_user.writeStream\
  .format("delta")\
  .outputMode("append")\
  .option("checkpointLocation", "abfss://silver@storagespotifyproject96.dfs.core.windows.net/DimUser/checkpoint")\
  .trigger(once=True)\
  .option("path","abfss://silver@storagespotifyproject96.dfs.core.windows.net/DimUser/data")\
  .toTable("spotify_catalog.silver.DimUser")

# DimArtist

In [0]:
df_art = spark.readStream\
  .format("cloudFiles")\
  .option("cloudFiles.format", "parquet")\
  .option("cloudFiles.schemaLocation", "abfss://silver@storagespotifyproject96.dfs.core.windows.net/DimArt/checkpoint")\
  .load("abfss://bronze@storagespotifyproject96.dfs.core.windows.net/DimArtist")

In [0]:
df_art_obj = reusable()
df_art = df_art_obj.dropColumns(df_art,['_rescued_data'])
df_art = df_art.dropDuplicates(['artist_id'])
display(df_art)

In [0]:
df_art.writeStream\
  .format("delta")\
  .outputMode("append")\
  .option("checkpointLocation", "abfss://silver@storagespotifyproject96.dfs.core.windows.net/DimArt/checkpoint")\
  .trigger(once=True)\
  .option("path", "abfss://silver@storagespotifyproject96.dfs.core.windows.net/DimArt/data")\
  .toTable("spotify_catalog.silver.DimArtist")

# DimTrack

In [0]:
df_track = spark.readStream\
  .format("cloudFiles")\
  .option("cloudFiles.format", "parquet")\
  .option("cloudFiles.schemaLocation", "abfss://silver@storagespotifyproject96.dfs.core.windows.net/DimTrack/checkpoint")\
  .load("abfss://bronze@storagespotifyproject96.dfs.core.windows.net/DimTrack")

In [0]:
display(df_track)

In [0]:
df_track = df_track.withColumn("duration_flag",when(col("duration_sec")<150,"low") \
                                               .when(col("duration_sec")<300,"medium")\
                                               .otherwise("high"))

                
df_track = df_track.withColumn("track_name",regexp_replace(col("track_name"),'-',' '))

df_track = reusable().dropColumns(df_track,['_rescued_data'])
    

In [0]:
display(df_track)

In [0]:
df_track.writeStream\
  .format("delta")\
  .outputMode("append")\
  .option("checkpointLocation", "abfss://silver@storagespotifyproject96.dfs.core.windows.net/DimTrack/checkpoint")\
  .trigger(once=True)\
  .option("path", "abfss://silver@storagespotifyproject96.dfs.core.windows.net/DimTrack/data")\
  .toTable("spotify_catalog.silver.DimTrack")

# DimDate

In [0]:
df_date = spark.readStream\
  .format("cloudFiles")\
  .option("cloudFiles.format", "parquet")\
  .option("cloudFiles.schemaLocation", "abfss://silver@storagespotifyproject96.dfs.core.windows.net/DimDate/checkpoint")\
  .load("abfss://bronze@storagespotifyproject96.dfs.core.windows.net/DimDate")

In [0]:
df_date = reusable().dropColumns(df_date, ["_rescued_data"])

In [0]:
display(df_date)

In [0]:
df_date.writeStream\
  .format("delta")\
  .outputMode("append")\
  .option("checkpointLocation", "abfss://silver@storagespotifyproject96.dfs.core.windows.net/DimDate/checkpoint")\
  .trigger(once=True)\
  .option("path", "abfss://silver@storagespotifyproject96.dfs.core.windows.net/DimDate/data")\
  .toTable("spotify_catalog.silver.DimDate")

# FactStream

In [0]:
df_fact = spark.readStream\
  .format("cloudFiles")\
  .option("cloudFiles.format", "parquet")\
  .option("cloudFiles.schemaLocation", "abfss://silver@storagespotifyproject96.dfs.core.windows.net/FactStream/checkpoint")\
  .load("abfss://bronze@storagespotifyproject96.dfs.core.windows.net/FactStream")

In [0]:
display(df_fact)

In [0]:
df_fact = reusable().dropColumns(df_fact, ["_rescued_data"])
df_fact.writeStream\
  .format("delta")\
  .outputMode("append")\
  .option("checkpointLocation", "abfss://silver@storagespotifyproject96.dfs.core.windows.net/FactStream/checkpoint")\
  .trigger(once=True)\
  .option("path", "abfss://silver@storagespotifyproject96.dfs.core.windows.net/FactStream/data")\
  .toTable("spotify_catalog.silver.FactStream")